In [1]:
import polars as pl
import numpy as np
import os

directory = r'D:\github\localt20\data\aftermerging'

team12Stats = pl.read_csv(os.path.join(directory, 'team12Statsflip.csv')).sort("match_id")
balltoball = pl.read_csv(os.path.join(directory, 'balltoballflip.csv')).sort("match_id")
playerStats = np.load(os.path.join(directory, 'matchPlayerStats.npy'), allow_pickle=True)
print(playerStats.shape, team12Stats.shape, balltoball.shape)
team12Stats

(1200, 22, 23) (6798, 24) (1686378, 6)


match_id,gender,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO,Mat_team2,Won_team2,Lost_team2,Tied_team2,NR_team2,W/L_team2,Inns_team2,HS_team2,LS_team2,AveRPW_team2,AveRPO_team2
i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
211028,1,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44
211028,1,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95
211048,1,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7
211048,1,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5
222678,1,3,1,2,0,0,0.5,3,201,114.0,18.66,7.72,2,1,0,1,0,1.0,2,134,0.0,20.0,6.84
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1451292,0,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42
1451293,0,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82
1451293,0,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42


In [3]:
len(np.unique(playerStats[:, 0, 0])), len(np.unique(team12Stats['match_id'])), len(np.unique(balltoball['match_id']))

(1200, 3399, 3683)

In [4]:
# Extract match_id columns from each dataset
team12_match_ids = team12Stats.select('match_id').unique()  # Polars unique match_ids in team12Stats
balltoball_match_ids = balltoball.select('match_id').unique()  # Polars unique match_ids in balltoball

# Extract match_ids from playerStats (assuming match_id is in the first column)
player_match_ids = np.unique(playerStats[:, 0, 0])

# Convert Polars Series to Numpy for intersection
team12_match_ids_np = team12_match_ids.to_numpy().flatten()
balltoball_match_ids_np = balltoball_match_ids.to_numpy().flatten()

# Find intersection of match_id across all three datasets
common_match_ids = np.intersect1d(np.intersect1d(team12_match_ids_np, balltoball_match_ids_np), player_match_ids)

print("Common match IDs across all datasets:", common_match_ids, len(common_match_ids))

Common match IDs across all datasets: [ 211028.  211048.  225263. ... 1450753. 1450759. 1450765.] 1156


In [5]:
# Filter team12Stats
filtered_team12Stats = team12Stats.filter(pl.col('match_id').is_in(common_match_ids))

# Filter balltoball
filtered_balltoball = balltoball.filter(pl.col('match_id').is_in(common_match_ids))

# Filter playerStats (assuming match_id is the first column)
filtered_playerStats = playerStats[np.isin(playerStats[:, 0, 0], common_match_ids)]

In [6]:
filtered_playerStats.shape, filtered_team12Stats.shape, filtered_balltoball.shape

((1156, 22, 23), (2312, 24), (534910, 6))

In [7]:
filtered_balltoball

match_id,innings,ball,curr_score,curr_wickets,won
i64,i64,f64,i64,i64,i64
211028,1,0.1,0,0,1
211028,1,0.2,1,0,1
211028,1,0.3,1,0,1
211028,1,0.4,1,0,1
211028,1,0.5,1,0,1
…,…,…,…,…,…
1450765,2,19.3,100,9,0
1450765,2,19.4,100,9,0
1450765,2,19.5,104,9,0


In [16]:
ps = np.array(filtered_playerStats[:,11:,:].tolist() 
             + filtered_playerStats[:,:11,:].tolist()
             + filtered_playerStats[:,:11,:].tolist()
             + filtered_playerStats[:,11:,:].tolist())
ps = ps.reshape(-1,22,23)
ps.shape

(2312, 22, 23)

In [17]:
# Save filtered datasets
filtered_team12Stats.write_csv(os.path.join(directory, '../filteredData/team12Stats.csv'))
filtered_balltoball.write_csv(os.path.join(directory, '../filteredData/balltoball.csv'))
np.save(os.path.join(directory, '../filteredData/playerStats.npy'), ps)